In [1]:
from decoder import *

2023-06-18 09:33:04.052993: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
import numpy as np

import random
import os
import sys
from numpy import arange
import math
import pyecharts
import sys,base64,urllib,re
import warnings 
from optparse import OptionParser
import logging
import logging.config
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow.keras.layers as layers

import time
import matplotlib.pyplot as plt
import jieba

# 加载分类数据
from tqdm import tqdm
corpus = []
MAX_LENGTH = 100
cat_name_dic = {
    '100': '民生',
    '101': '文化',
    '102': '娱乐',
    '103': '体育',
    '104': '财经',
    '106': '房产',
    '107': '汽车',
    '108': '教育',
    '109': '科技',
    '110': '军事',
    '112': '旅游',
    '113': '国际',
    '114': '证券',
    '115': '农业',
    '116': '电竞'
}
with open('./toutiao_cat_data.txt', 'r', encoding='utf-8') as f:
    for k in tqdm(f):
        new_id, cat, cat_n, title, title_kws = k.strip("").split("_!_")
        cat_name = cat_name_dic.get(cat, '')
        if cat_name == "":
            continue
        title = title.replace("|", "")
        title = title + "|" + cat_name
        if len(title) > MAX_LENGTH:
            continue
        title = " ".join(jieba.cut(title, cut_all=False))
        corpus.append(title)

0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.611 seconds.
Prefix dict has been built successfully.
382688it [00:50, 7632.74it/s]


In [3]:
corpus_format = []
for k in tqdm(corpus):
    k = k.split(" ")
    corpus_format.append(" ".join(k))
    
random.shuffle(corpus_format)
train_examples = corpus_format

tokenizer_title = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (k for k in train_examples), target_vocab_size=20000)


100%|██████████| 382675/382675 [00:00<00:00, 515393.78it/s]


In [4]:
def encode(lang):
    lang = [tokenizer_title.vocab_size] + tokenizer_title.encode(lang) + [tokenizer_title.vocab_size + 1]
    return lang

def pad_with_zero(lang, max_length=MAX_LENGTH):
    n = MAX_LENGTH - len(lang)
    lang = lang + [0 for k in range(n)]
    return lang

In [5]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64
train_examples = corpus_format[:300000]
train_examples = [encode(k) for k in train_examples]
train_examples = [k for k in train_examples if len(k) <= MAX_LENGTH]
train_examples = [pad_with_zero(k) for k in train_examples]
train_dataset = tf.data.Dataset.from_tensor_slices(train_examples)

# 使用缓存数据加速读入
train_dataset = train_dataset.cache()

# 打乱并获取批数据
train_dataset = train_dataset.batch(BATCH_SIZE)

# 设置预取数据
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

2023-06-18 09:37:44.437039: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-18 09:37:44.476612: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:3f:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.77GiB deviceMemoryBandwidth: 707.88GiB/s
2023-06-18 09:37:44.476645: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-18 09:37:44.479863: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-18 09:37:44.479938: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-06-18 09:37:44.480837: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuff

In [6]:
# 构建掩码
def create_mask(targets):

    # look_ahead 掩码， 掩掉未预测的词
    look_ahead_mask = create_look_ahead_mark(tf.shape(targets)[1])
    
    # 解码层第一层得到padding掩码
    decode_targets_padding_mask = create_padding_mark(targets)

    # 合并解码层第一层掩码
    combine_mask = tf.maximum(decode_targets_padding_mask, look_ahead_mask)

    return combine_mask

def loss_fun(y_ture, y_pred):
    mask = tf.math.logical_not(tf.math.equal(y_ture, 0))  # 为0掩码标1
    loss_ = loss_object(y_ture, y_pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [7]:
from gpt2 import GPT2
from decoder import Decoder

In [8]:
num_layers = 4
d_model = 128
dff = num_layers * d_model
num_heads = 8
target_vocab_size = tokenizer_title.vocab_size + 2
max_seq_len = MAX_LENGTH
dropout_rate = 0.1

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

# 定义优化器
learing_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learing_rate, beta_1=0.9, 
                                     beta_2=0.98, epsilon=1e-9)

# 定义目标函数和评估指标
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                           reduction='none')

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# 定义模型
gpt2 = GPT2(num_layers, d_model, num_heads, dff,
            target_vocab_size,
            max_seq_len, 
            dropout_rate)

checkpoint_path = './checkpoint/train_gpt2_exp1'
ckpt = tf.train.Checkpoint(gpt2=gpt2,
                          optimizer=optimizer)
# ckpt管理器
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=3)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('last checkpoit restore')

In [10]:
def train_step(targets):
    tar_inp = targets[:, :-1]
    tar_real = targets[:, 1:]
    
    # 构造掩码
    combined_mask = create_mask(tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = gpt2(tar_inp, True, combined_mask)
        loss = loss_fun(tar_real, predictions)
        
    # 求梯度
    gradients = tape.gradient(loss, gpt2.trainable_variables)
    
    # 反向传播
    optimizer.apply_gradients(zip(gradients, gpt2.trainable_variables))

    # 记录loss和准确率
    train_loss(loss)
    train_accuracy(tar_real, predictions)

EPOCHS = 3
step_list = []
loss_list = []
step = 0

for epoch in range(EPOCHS):
    start = time.time()

    # 重置记录项
    train_loss.reset_states()
    train_accuracy.reset_states()

    for batch, all_inputs in enumerate(train_dataset):
        
        # 训练
        train_step(all_inputs)

        if batch % 100 == 0:
            loss = train_loss.result()
            print('epoch {}, batch {}, loss:{:.4f}, acc:{:.4f}'.format(
                epoch+1, batch, loss, train_accuracy.result()
            ))
            step_list.append(step)
            loss_list.append(loss)
        step += 1

    if (epoch + 1) % 2 == 0:
        ckpt_save_path = ckpt_manager.save()
        print('epoch {}, save model at {}'.format(
        epoch+1, ckpt_save_path
        ))


    print('epoch {}, loss:{:.4f}, acc:{:.4f}'.format(
        epoch+1, train_loss.result(), train_accuracy.result()
    ))

    print('time in 1 epoch:{} secs\n'.format(time.time()-start))

epoch 1, batch 0, loss:1.6026, acc:0.0177
epoch 1, batch 100, loss:1.4949, acc:0.0166
epoch 1, batch 200, loss:1.4649, acc:0.0167
epoch 1, batch 300, loss:1.4361, acc:0.0174
epoch 1, batch 400, loss:1.4098, acc:0.0189
epoch 1, batch 500, loss:1.3874, acc:0.0201
epoch 1, batch 600, loss:1.3668, acc:0.0214
epoch 1, batch 700, loss:1.3520, acc:0.0224
epoch 1, batch 800, loss:1.3386, acc:0.0234
epoch 1, batch 900, loss:1.3265, acc:0.0243
epoch 1, batch 1000, loss:1.3154, acc:0.0251
epoch 1, batch 1100, loss:1.3053, acc:0.0258
epoch 1, batch 1200, loss:1.2954, acc:0.0266
epoch 1, batch 1300, loss:1.2864, acc:0.0273
epoch 1, batch 1400, loss:1.2778, acc:0.0280
epoch 1, batch 1500, loss:1.2691, acc:0.0286
epoch 1, batch 1600, loss:1.2615, acc:0.0293
epoch 1, batch 1700, loss:1.2542, acc:0.0299
epoch 1, batch 1800, loss:1.2477, acc:0.0305
epoch 1, batch 1900, loss:1.2404, acc:0.0312
epoch 1, batch 2000, loss:1.2335, acc:0.0317
epoch 1, batch 2100, loss:1.2267, acc:0.0323
epoch 1, batch 2200, l

In [11]:
def evaluate(inp_sentence):
    start_token = [tokenizer_title.vocab_size]
    end_token = [tokenizer_title.vocab_size + 1]
    
    # 增加开始和结束标记
    inp_sentence = start_token + tokenizer_title.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    decoder_input = [tokenizer_title.vocab_size]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(MAX_LENGTH):
        
        combined_mask = create_mask(encoder_input)
        
        predictions, _ = gpt2(encoder_input, False, combined_mask)

        # 从 seq_len 维度选择最后一个词
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 如果 predicted_id 等于结束标记，就返回结果
        if predicted_id == tokenizer_title.vocab_size + 1:
            return tf.squeeze(encoder_input, axis=0)

        # 连接 predicted_id 与输出，作为解码器的输入传递到解码器。
        encoder_input = tf.concat([encoder_input, predicted_id], axis=-1)
        output = tf.concat([output, predicted_id], axis=-1)
    
    return tf.squeeze(encoder_input, axis=0)
    
def translate(sentence, plot=''):
    result = evaluate(sentence)

    predicted_sentence = tokenizer_title.decode([i for i in result if i < tokenizer_title.vocab_size]) 
    predicted_sentence = predicted_sentence.replace(" ", "")
    sentence = sentence.replace(" ", "")

    print('输入: {}'.format(sentence))
    print('预测输出: {}'.format(predicted_sentence))

In [13]:
test = corpus_format[300000:]
i = 0
print("真实数据：", test[i].replace(" ", ""))
translate(test[i][:-4])
print("============================")
i = 1
print("真实数据：", test[i].replace(" ", ""))
translate(test[i][:-4])
print("============================")
i = 2
print("真实数据：", test[i].replace(" ", ""))
translate(test[i][:-4])
print("============================")
i = 3
print("真实数据：", test[i].replace(" ", ""))
translate(test[i][:-4])

真实数据： 当你的车离合线断了，汽修厂让你自己开过去，你该怎么做才能把车送到汽修厂？|汽车
输入: 当你的车离合线断了，汽修厂让你自己开过去，你该怎么做才能把车送到汽修厂？
预测输出: 当你的车离合线断了，汽修厂让你自己开过去，你该怎么做才能把车送到汽修厂？的问题？|汽车
真实数据： 葡媒：国米、拉齐奥关注波尔图主帅孔塞桑|体育
输入: 葡媒：国米、拉齐奥关注波尔图主帅孔塞桑
预测输出: 葡媒：国米、拉齐奥关注波尔图主帅孔塞桑训练|体育
真实数据： 今日将爆发最强内战，天津权健vs广州恒大，阿兰能否带领恒大取胜|体育
输入: 今日将爆发最强内战，天津权健vs广州恒大，阿兰能否带领恒大取胜
预测输出: 今日将爆发最强内战，天津权健vs广州恒大，阿兰能否带领恒大取胜进球？|体育
真实数据： 为什么说历史不忍细读呢？|文化
输入: 为什么说历史不忍细读呢？
预测输出: 为什么说历史不忍细读呢？的是什么原因？|文化


In [17]:
s = " ".join(list(jieba.cut("小麦收成")))
translate(s)
print("============================")
s = " ".join(list(jieba.cut("整容狂人")))
translate(s)
print("============================")
s = " ".join(list(jieba.cut("北大校长口误")))
translate(s)
print("============================")


输入: 小麦收成
预测输出: 小麦收成，玉米的价格会怎么样？|农业
输入: 整容狂人
预测输出: 整容狂人的女星，网友：这是你的最爱的吗？|娱乐
输入: 北大校长口误
预测输出: 北大校长口误的孩子们都在干什么？|教育
